# Kubernetes Tutorial

**Note:** You can complete this tutorial with a partner if you have trouble downloading/installing Docker Desktop or any of the Kubernetes software.

<a id='prereq'></a>
## Prerequisites
1. Install the Kubernetes command-line tool, [kubectl](https://kubernetes.io/docs/tasks/tools/install-kubectl/), so that you can communicate with Kubernetes clusters.
    1. Follow the instructions in the link to install the proper version depending on your operating system (Linux, MacOS, or Windows). You should use version `v1.16.0`, but if you can't run it, don't worry about it.

## 1. Build a Docker image

We will now put our sentiment analysis Flask application into a Docker container.

3. Let's create a Docker container for our Flask application.
    2. Run `docker build -t bhavenp/sentiment-analysis:latest -f Dockerfile .`
    3. Run `docker image ls` to see that we created a new Docker image with the tag `server`.
    4. Run `docker run -it --rm -p 5000:5000 bhavenp/sentiment-analysis:latest /bin/bash ml_deploy_demo/run.sh`.
        2. By using the `-p 5000:5000` flag, we expose port 8080 to the outside so our web page can be accessed.
4. Use Postman to hit the endpoint at http://0.0.0.0:5000/predict.

5. Clean-up steps:
    1. Stop your containers with `docker stop $(docker container ls -q)`. This may take a few seconds.
    2. Delete your containers with `docker rm $(docker ps -aq)`. Beware, this will stop all containers you have running on your computer, so if you have containers running for other classes, you will have to remove the containers using the container IDs.
    3. Use `docker rmi <IMAGE_ID>` to delete your images, if you would like to.

## 2. Deploy your container to a Kubernetes Cluster

### Prerequites
You will need to have installed `kubectl`, *AWS CLI* and `eksctl`.

### Create Cluster
1. Run `eksctl create cluster -f cluster.yaml`.
    1. This will take 10-15 minutes.
2. Once this is done, run `kubectl get nodes` to make sure your `kubectl` is configured to communicate with the cluster you just created.
    1. You should see 3 IP addresses with **Status** as *Ready*.
    
### Deploy *hello world* application
3. We can create a deployment for our Docker image using `kubectl create -f sentiment_analysis_deployment.yaml`.
    1. After a few seconds, run `kubectl get po` to see the pod you just created for your Flask application. The **Status** should saying *Running*. If the **Status** is not *Running*, wait a few more seconds and run `kubectl get po` again.
4. We can create a service for our running pod using `kubectl create -f sentiment_analysis_service.yaml`.
    1. After a few seconds, run `kubectl get svc` to see the service you just created for your Flask application.
    2. You should see a link under **EXTERNAL-IP** that looks similar to  
    `ab0ffcc67371d11eabbec0249b21ade2-2082982831.us-east-1.elb.amazonaws.com`.
5. Open a web browser and go to `http://<link_from_step_B>:5000/`. You should see "Hello world!", possibly in a different language. If you refresh the page a few times, the text should change to "Hello world!" in different languages.

### Clean-up app and delete cluster
5. Run `kubectl delete -f sentiment_analysis_service.yaml` to delete the service. This deletes the *LoadBalancer*, which prevents anyone from accessing the application from the outside world.
    1. Verify this by running `kubectl get svc`. You should no longer see the *hello-world-server-service*.
6. Run `kubectl delete -f sentiment_analysis_deployment.yaml` to delete the pod holding our Docker container. Our application is no longer running on the Kubernetes cluster.
    1. Verify this by running `kubectl get po`. You may see that the **STATUS** of your pod is *Terminating* or a message saying "*No resources found in default namespace.*".
7. Run `eksctl delete cluster -f cluster.yaml` to delete your EKS cluster.
    1. This will take 10-15 minutes.

# OLD instructions

## 3. Deploy your container to a Kubernetes Cluster

### Prerequites
You will need to have installed `kubectl` and `Minikube` according to the instructions [above](#prereq).  
This part of the Kubernetes tutorial was adapted from https://medium.com/@yzhong.cs/getting-started-with-kubernetes-and-docker-with-minikube-b413d4deeb92.

1. Run `minikube start` in your Terminal window.
    1. You should see several lines beginning with emojis. The last line should say `Done! kubectl is now configured to use "minikube"`.
    2. You now have a virtual cluster running on your machine/computer. You can think of this cluster as running a virtual machine on your personal computer.

2. Kubernetes organizes Docker containers into [Pods](https://kubernetes.io/docs/concepts/workloads/pods/pod/). Docker containers in the same pod share CPU allocation and memory. Typically, you would want multiple Docker containers in the same pod because they must interact to achieve some process, such as dealing with reads and writes to a database. [Some info on why a pod may have multiple containers](https://linchpiner.github.io/k8s-multi-container-pods.html).
    1. To create a pod, we must also create a [Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/), which is basically a set of rules for how much CPU and memory a pod should have access to and different labels/names for a pod. Additionally, a deployment specifies what should happen to a pod if it stops running.
    2. Deployments can be configured from the command-line, but this becomes difficult when you have many parameters to specify. Therefore, users typically generate a YAML file specifying the details of their deployment, which is what we will do as well. It is convention to name the deployment object the same as the YAML file.
        1. Please download the `hello_world_db_deployment.yaml` file from the Lecture 25 page on the course website and place it in your `L25/` directory.
        2. The name of our Deployment is `hello-world-db-deployment`.
        3. We only want one pod for our deployment, as indicated by `replicas: 1`.
        4. The `selector` defines how the Deployment finds which Pod(s) to manage. In this case, we simply select a label that is defined in the Pod template. That’s what the two books-app fields are for.
        5. We specify the Docker image and version we want to use- `bhavenp/cs207-lecture25:db`. The `imagePullPolicy` is set to `Always` since we want to pull the Docker image from Docker Hub whenever we create a new Pod. I already pushed our database server Docker image to a repository on Docker hub, making it easier for us to deploy the Docker image with our web application using Kubernetes.
        6. You can see that we expose port 8081 for the Pod.
    2. We can create a deployment for our Docker image using `kubectl create -f hello_world_db_deployment.yaml`. 
    3. You should get a message saying "deployment.apps/hello-world created".

3. You can now use the `kubectl get deployments` command to see that your deployment is available, meaning it is ready to receive HTTP requests. Use `kubectl get pods` command to see that pods are running.

Now, the Pod is running our database server running on our Kubernetes cluster, however this server is not exposed to the outside world because it is encapsulated within the cluster. Check this by trying to connect to http://localhost:8081/. You should get a connection error. We need to now create a deployment for our front-end web server, so that we can access our web application.

4. Let's create a deployment for our front-end server.
    1. Please download the `hello_world_server_deployment.yaml` file from the Lecture 25 page on the course website and place it in your `L25/` directory.
    2. Remember that we need to have our front-end server connect to our database server, like we accomplished when we were just running the Docker containers on our local machines. To accomplish this, we need to know the internal IP address that Kubernetes has assigned to the Pod running our database server. This IP address can only be used from within the Kubernetes cluster. We can get this IP by running `kubectl get pods -o=custom-columns=NAME:.metadata.name,IP:.status.podIP`.
    3. Open up the `hello_world_server_deployment.yaml` file and make sure the IP you get for the `hello-world-db-deployment` is the same IP address specified in the `value` field for the `DB_URL`. If it is the same, then you're good to go. If it is different, then please change it.
        1. You will also see that we specify port 8080 as a communication port for the Pod.
    4. Now, we can run ` kubectl create -f hello_world_server_deployment.yaml` to create a deployment for our front-end web server.
    5. Use `kubectl get deployments` and `kubectl get pods` to see that your pods are running.

6. Now that we have both parts of our application running, try visiting http://localhost:8080/. Can you access the web application? The answer should be no because although we have exposed ports for our Pods, they are not accessible to the outside world. So far, we can only communicate with the Pods running our application if we are within the Kubernetes cluster (again, this is for isolation purposes). Let's check this out from within the cluster.
    1. Run `kubectl get pods -o=custom-columns=NAME:.metadata.name,IP:.status.podIP` and copy the IP address for the `hello-world-server-deployment`.
    2. Run `minikube ssh` in your Terminal window. You should see "minikube" written in you Terminal window.
    3. Run `curl <hello-world-server-deployment-URL>:8080/` several times. You should get responses for "Hello world!" in different languages, demonstrating that the application is working.
    4. Run `exit` to exit the minikube Terminal.

7. To be able to access our web application from outisde, we need to create a Kubernetes [Service](https://kubernetes.io/docs/concepts/services-networking/service/) to make the `hello-world` container accessible from outside the Kubernetes virtual network.
    1. Use `kubectl expose deployment hello-world-server-deployment --type=LoadBalancer --port=8080` to allow your container to receive HTTP request from outside.
    2. You should get a message that says "service/hello-world exposed".
    
8. You can view the status of your sercice by using the `kubectl get services` command.
    1. Notice that the `EXTERNAL-IP` for our service is *pending*. If we were running our Kubernetes cluster using a cloud provider, such as AWS, we would get an actual IP address to access our service from anywhere.
    2. Since we are running Minikube, you can access your service by using the `minikube service hello-world-server-deployment`. This should automatically open a web page with our `Hello world!` page. Reload this page a few times to see the different "Hello world!" translations.
    
9. Congratulations! You have deployed a web application using Kubernetes!

Below is a schematic of your Dockerized web application on a Kubernetes (K8s) cluster. The two servers are running in individual Docker containers within the Kubernetes cluster, which is emulated by a VM in your case. Generally, a Kubernetes cluster would exists across serveral machines on a cloud provider like AWS. The K8s cluster takes care of generating a network for us and provides individual IP addresses for each of our Pods, reducing our work. By creating a K8s service for your web application, you provide outside users an entry point to use your application while protecting your database.
![](../fig/K8s_python_app.png)

    
10. You can clean up the resources and cluster using:
    1. `kubectl delete service hello-world-server-deployment`
    2. `kubectl delete deployment hello-world-server-deployment`
    2. `kubectl delete deployment hello-world-db-deployment`
    3. `minikube stop`
    3. `minikube delete`